- Find the oldest product
- Find the newest product
- Average product age, where age means how long the product has been in the system.
- List of other countries where products are sold too.
- Identify category of products and the compute:
   Number of products by category
    List containing names of products by category 
- Identify traces and compute:
    Number of products by trace
    List containing names of products by trace
- Data quality analysis on fields of interest (see appendix 1):
   Number of products with complete info.
    % of products without complete analysis per 100g
    % of products without additives info
    % of products without traces info
- Data profiling on fields of interest (see appendix 1):
    Stats on analysis per 100g fields

In [1]:
#Importing APIs
import pandas as pd
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from pyspark.sql.window import Window
from IPython.display import display, Markdown
from pyspark.sql import SQLContext


sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sqlContext = SQLContext(spark)

In [2]:
#Load dataset
resto = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", "\t") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "false") \
                 .csv("noT.csv")
tt = spark.read \
                 .option("inferSchema", "true") \
                 .option("delimiter", ";") \
                 .option("mode","PERMISSIVE") \
                 .option("header", "true") \
                 .csv("title.csv")

In [3]:
resto.show(3)
rest = resto.drop("_c172","_c173","_c174","_c175")
#.drop("C_173").drop("C_174").drop("C_175")
rest.show(1)
tt.show(1)
concate = tt.union(rest)

+--------------------+----------------+-----+---------+---------+---+------------------+------------------+--------------------+--------------------+--------------------+----+----+----+-------+-------+--------------------+--------------------+--------------------+----+----+----+----+----+----+----+-----------+----------+-------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+-----+----+----+----+--------------------+--------------------+----+----+----+----+----+----+----+----+----+-------------+-------+--------------------+--------------------+--------------------+----+--------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-----+------+----+----+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----

In [4]:
concate.show(2)

+--------------------+---+-------+---------+---------------+------------+------------------------+------------------+-----------+--------------+--------------+--------------+------+-----------+----------+---------------+-------------+----------------+--------------------+-------------------------+------+-----------+---------+--------------+------+-----------+---------------+----------+---------+----------------+---------+--------------+------+-----------+------------+----------------+-------------+-----------+--------------+---------------------------+-------------------------+------------------------------+---------------------------------------+-------------------------------------+------------------------------------------+----------------+----------------+----------+-------------+-------------+------+-----------+-------------+--------------------+--------------------+--------------------+---------------------+---------------------+---------------------------+--------------------+--

In [22]:
#checking the right position of the columns
concate.filter(col("Ingredients_text") != "").select("Ingredients_text").show(25)

+--------------------+
|    Ingredients_text|
+--------------------+
|                  ""|
|                  ""|
|Zucker, Kakaomass...|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|                  ""|
|Zucker,Palmöl, Ha...|
|             Germany|
|             Germany|
|Proteinmischung (...|
|Proteinmischung (...|
|100% Soja-Protein...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|Molkenproteinkonz...|
|sugar, corn syrup...|
|sugar, corn syrup...|
|                  ""|
+--------------------+
only showing top 25 rows



Creator
Created_datetime
Last_modified_datetime
Product_name
Countries_en
Traces_en
Additives_tags
Main_category_en
Image_url
Quantity
Packaging_tags
Categories_en
Ingredients_text
Additives_en
Energy-kcal_100g
Fat_100g
Saturated-fat_100g
Sugars_100g
Salt_100g/sodium_100g

In [23]:
prepared_df = concate.select("Creator",
"Last_modified_t",
"Product_name",
"Countries",
"Traces",
"Additives_tags",
"Main_category",
"Image_url",
"Quantity",
"Packaging_tags",
"Categories",
"Ingredients_text",
"Additives_n",
"Energy-kcal_100g",
"Fat_100g",
"Saturated-fat_100g",
"Sugars_100g",
"Salt_100g")
#note: could not find additive, replaced with additive_n

In [25]:
prepared_df.show(3)

+-------+---------------+------------+---------+--------------------+--------------+-------------+--------------------+--------------------+--------------------+----------+--------------------+-----------+--------------------+--------+------------------+-----------+---------------+
|Creator|Last_modified_t|Product_name|Countries|              Traces|Additives_tags|Main_category|           Image_url|            Quantity|      Packaging_tags|Categories|    Ingredients_text|Additives_n|    Energy-kcal_100g|Fat_100g|Saturated-fat_100g|Sugars_100g|      Salt_100g|
+-------+---------------+------------+---------+--------------------+--------------+-------------+--------------------+--------------------+--------------------+----------+--------------------+-----------+--------------------+--------+------------------+-----------+---------------+
|     ""|      barquette|          ""|  Germany|                  ""|            ""|      unknown|en:to-be-complete...|                  ""|           

# Subset data analysis in Spark

## Find the oldest product

# Specifc analysis 